# Solving Dynamical Systems using Reservoir Computing

In [1]:


from QRC.solvers import *
from QRC.systems import *
from QRC.post_process import *
from QRC.crc import *
from QRC.qrc import *
from QRC.validation import *

import h5py
from datetime import date
today = date.today()
import sklearn 
import numpy as np
import scipy
from scipy.sparse import csr_matrix, csc_matrix, lil_matrix
from scipy.sparse.linalg import eigs as sparse_eigs
import skopt


#plotting libraries
import matplotlib.pyplot as plt
import matplotlib.pylab as pl
import matplotlib as mpl
mpl.rc('text', usetex = True)
mpl.rc('font', family = 'serif')
mpl.rcParams['text.usetex']

#scipy libraries
from skopt.space import Real
from skopt.learning import GaussianProcessRegressor as GPR
from skopt.learning.gaussian_process.kernels import Matern, WhiteKernel, Product, ConstantKernel
import matplotlib as mpl
import time
from skopt.plots import plot_convergence
import scipy.stats as stats 

#QISKIT
from qiskit import  QuantumCircuit, assemble, QuantumRegister, ClassicalRegister, transpile
import random as rnd
import time
from qiskit.visualization import plot_histogram
from qiskit.circuit import ParameterVector, Parameter

from timeit import default_timer as timer


#TODO
# Add config file so that to run on HPC


# Select Systems and Parameters

In [2]:
#for Lorenz 63:
if __name__ == "__main__":
    dt = 0.01
    tot_steps = 100000
    upsample = 1
    q0 =  np.array([7.432487609628195, 10.02071718705213, 29.62297428638419])
    #q0 =  np.array([0, 1, 1])
    N_transient = 2000


save_fold = 'VPT_L63'
system = Systems(dt,tot_steps,q0,upsample,N_transient)

dim,N_lyap,N_t = system.set_param_lorenz63() # FOR MFE, N_t should be input so in set_param_MFE -- make it as input
q = system.gen_data_lorenz63()

# Pre-process input data 

In [3]:
data_inputs = True
noise       = False # to add noise in flattened time series 
scaling     = True # to scale flattened time series 

if data_inputs:
    N_ts        = 1 #only first N_ts used for training and validation
    N_test_stat = 500 # Time series for testing after N_ts

    N_washout  = 2*N_lyap #10 before for L96 10 D , 2 before for L63 / MFE  
    N_val      = 5*N_lyap # 3 for L63 , # 5 for L96 , 2*N_lyap for MFE
    N_train    = 20*N_lyap #200 before for L96 10D , 20 before for L63 / MFE  
    N_test     = 200*N_lyap
 


# All above in this block are Inputs
q_unscaled = q
UU = q # backup of complete time series
print('Laminarized precentage', np.round(100-((q.shape[0])*100/N_t),2),'(relevant for MFE)')

q = q[:N_ts+N_test_stat,:,:] # trimmed time series

print('Retaining ',N_ts, 'series for training and validation')
print('Additional ',N_test_stat, 'series for testing and statistical predictions')

# Rescaling Input Time Series after flattening it (only retained series)
q0 = q.shape[0] # Total time series, Train+Val+Test
q1 = q.shape[1] # Length of each time series
q_U = q.reshape(q0*q1, dim)

if scaling:
    q = sklearn.preprocessing.minmax_scale(q_U, feature_range=(0, 1), axis=0, copy=True) # rescaling flattened array
    q  = q.reshape(q0,q1,dim) # making 3 dimensional again

# from here q has scaled data
q_tv = q[:N_ts,:,:]  # From q retaining training set (N_ts)
q_test = q[N_ts:N_ts+N_test_stat,:,:]  #From q removing retaining test set(N_test_stat)

# Training, Validation Parameters
UU = q_tv # Training, validation set
N0 = UU.shape[0] # Retained Time series for training
N1 = UU.shape[1] # Length of each time series
print('UU Size =',UU.shape) # 3 dim
U = UU.reshape(N0*N1, dim) # Generating flattened time series for addition of noise and normalization
print('U Size =',U.shape) # 2 dim 

if noise: # adding noise in training set
    target_snr_db=40
    seed=0
    UU = add_noise(U,target_snr_db,seed,dim,N0,N1)

#compute norm
U_data = U[:N_washout+N_train+N_val]
m = U_data.min(axis=0)
M = U_data.max(axis=0)
norm = M-m
u_mean = U_data.mean(axis=0)

# washout
U_washout = UU[:,:N_washout]
# training + validation
U_tv  = UU[:,N_washout:N_washout+N_train+N_val-1]
Y_tv  = UU[:,N_washout+1:N_washout+N_train+N_val]
# Testing
U_test  = UU[:,N_washout+N_train+N_val:N_washout+N_train+N_val+N_test-1]
Y_test  = UU[:,N_washout+N_train+N_val+1:N_washout+N_train+N_val+N_test]

print('N_lyap:', N_lyap)
print('N_train =',int(N_train/N_lyap),'N_lyap',' x ',str(N_ts),'series')
print('N =',int((N_washout+N_val+N_train)/N_lyap),'N_lyap',', Total steps of training=',int(N_washout+N_val+N_train),', Out of = ',N0*N1)
print('The shape of q_tv is '+str(q_tv.shape)+ ' to train on ' +str(N_ts)+  ' time series')
print('The shape of q_test is '+str(q_test.shape)+ ' to test ' +str(N_test_stat)+  ' time series, and for stats predictions (relevant for MFE)')
print('The shape of q is '+str(q.shape)+ ' combining above')
print('The shape of U_test is '+str(U_test.shape)+ ' combining above')


Laminarized precentage 0.0 (relevant for MFE)
Retaining  1 series for training and validation
Additional  500 series for testing and statistical predictions
UU Size = (1, 98000, 3)
U Size = (98000, 3)
N_lyap: 111
N_train = 20 N_lyap  x  1 series
N = 27 N_lyap , Total steps of training= 2997 , Out of =  98000
The shape of q_tv is (1, 98000, 3) to train on 1 time series
The shape of q_test is (0, 98000, 3) to test 500 time series, and for stats predictions (relevant for MFE)
The shape of q is (1, 98000, 3) combining above
The shape of U_test is (1, 22199, 3) combining above


# Visualize input data

In [4]:
# l1 = 8000
# l2= 5000
# l3= 5000

# plot_lorenz63_attractor(U,l1)
# plot_lorenz63_time(U,N_lyap,l2,l3)
# plt.show()

# Classical Reservoir Initialization

In [5]:
# Required Input Parameters
bias_in   = np.array([np.mean(np.abs((U_data-u_mean)/norm))]) #input bias (average absolute value of the inputs)
bias_out  = np.array([1.]) #output bias 
N_units   = 512 #neurons
density = 0.11 # L63
#density = 0.01 # L96
#connectivity = 30

# Predefining Hyperparameter to initiate a class
tikh = np.array([1e-9])  # Tikhonov factor (optimize among the values in this list) #TODO
sigma_in = 1 # input scaling
rho = 1
epsilon = 1

# Random Weight Matrices Seed
seed = 2

sparseness = 1 - density
connectivity = (1 -  sparseness)*(N_units-1)

# sparseness = 1-((connectivity)/(N_units-1))
# density    = 1-sparseness
print('Density(D) = ',np.round(density,2))
print('Connectivity = ', np.round(connectivity,2))

esn = EchoStateNetwork(tikh,sigma_in,rho,epsilon,bias_in,bias_out,N_units,dim,density)
esn.norm_u = norm

Density(D) =  0.11
Connectivity =  56.21


# Validation (hyperparameter tuning)

In [6]:
n_in        = 10               #Number of Initial random points
n_tot       = 50               #Total number of function evaluations

#range for hyperparameters 
spec_in     = np.log10(0.01)   
spec_end    = np.log10(1.0)   
epsilon_in  = 0.01
epsilon_end = 1
scaling_in  = 0.01
scaling_end = 1
#Number of Networks in the ensemble
ensemble = 5      
    
search_space = [Real(spec_in, spec_end, name='spectral_radius'),
            Real(epsilon_in, epsilon_end, name='epsilon'),
            Real(scaling_in,scaling_end, name = 'input_scaling')]

kernell = ConstantKernel(constant_value=1.0, constant_value_bounds=(1e-1, 3e0)) *\
            Matern(length_scale=[0.2,0.1,0.3], nu=2.5, length_scale_bounds=(5e-2, 1e1)) 


k= 0
# Which validaton strategy (implemented in Val_Functions.ipynb)
val      = RVC_Noise_upt
N_fo     = max(50,2)            # number of validation intervals
N_in     = N_washout                  # timesteps before the first validation interval (can't be 0 due to implementation)
N_fw     = (N_train-N_val)//(N_fo-1) # how many steps forward the validation interval is shifted (in this way they are evenly spaced)

#Quantities to be saved
par      = np.zeros((ensemble, 5))      # GP parameters
x_iters  = np.zeros((ensemble,n_tot,len(search_space))) # coordinates in hp space where f has been evaluated
f_iters  = np.zeros((ensemble,n_tot))   # values of f at those coordinates
minimum  = np.zeros((ensemble, len(search_space)+2))      # minima found per each member of the ensemble

# to store optimal hyperparameters and matrices
tikh_opt = np.zeros(n_tot)
Woutt    = np.zeros(((ensemble, N_units+1,dim)))
Winn     = [] #save as list to keep single elements sparse
Ws       = [] 
biass    = []
rho_ens  = []
epsilon_ens = []
sigma_in_ens = []
Xa_cc        = []
LHS_cc       = []
RHS_cc       = []

# save the final gp reconstruction for each network
gps        = [None]*ensemble

# to print performance of every set of hyperparameters
# print_flag = False



In [7]:
## TODO fix k, global issues for classical hyperparam opt
run_hyperparam = False

if run_hyperparam:
    for i in range(ensemble):
        
        print('Realization    :',i+1)
        
        k   = 0
        
        # Win and W generation
        seed= i+1
        rnd = np.random.RandomState(seed)

        Win =  esn.gen_input_matrix(seed)
        W =  esn.gen_reservoir_matrix(seed)
        
        # Bayesian Optimization
        tt       = time.time()
        res      = validate(val,kernell,search_space,n_in,n_tot,esn,tikh,N_fo,N_fw,N_in,N_val,N_washout,U,U_washout,U_tv,Y_tv,Win,W,tikh_opt)
        print('Total time for the network:', time.time() - tt)
        
        #Saving Quantities for post_processing
        gps[i]     = res.models[-1]    
        gp         = gps[i]
        x_iters[i] = np.array(res.x_iters)
        f_iters[i] = np.array(res.func_vals)
        minimum[i] = np.append(res.x,[tikh_opt[np.argmin(f_iters[i])],res.fun])
        params     = gp.kernel_.get_params()
        key        = sorted(params)
        par[i]     = np.array([params[key[2]],params[key[5]][0], 1, gp.noise_,params[key[5]][1]])

        esn.rho      = 10**minimum[i,0]
        esn.epsilon  = minimum[i,1]
        esn.sigma_in = minimum[i,2]

        Woutt[i] = esn.train(U_washout, U_tv, Y_tv,Win,W)[1]

        Winn    += [Win] 
        Ws      += [W]   

        #Plotting Optimization Convergence for each network
        print('Best Results: x', 10**minimum[i,0], minimum[i,1],minimum[i,2] ,minimum[i,3],', Min Obj Function =', minimum[i,4])
        print('Hyperparamters:',esn.rho,esn.epsilon,esn.sigma_in)

        plt.rcParams["figure.figsize"] = (15,2)

# Loading Hyperparameters


In [8]:
#hf       = h5py.File('./data/VPT_L96/minimum='+str(dt)+'-Nres'+str(N_units)+'-Ensemble'+str(ensemble)+'-dim'+str(dim)) # L96 without noise , its the file for paper
# hf       = h5py.File('./data/VPT_L96/minimum='+str(dt)+'-Nres'+str(N_units)+'-Ensemble'+str(ensemble)+'-dim'+str(dim)+'-noise') # L96 with noise , paper results
#hf       = h5py.File('./data/VPT_L96/minimum='+str(dt)+'-Nres'+str(N_units)+'-Ensemble'+str(ensemble)+'-dim'+str(dim)+'-noise'+str(noise)+'new')

#hf       = h5py.File('./data/VPT_L63/minimum='+str(dt)+'-Nres'+str(N_units)+'-Ensemble'+str(ensemble)+'-noise')
# hf       = h5py.File('./data/VPT_L63/minimum='+str(dt)+'-Nres'+str(N_units)+'-Ensemble'+str(ensemble)+'-noise'+str(noise))
hf       = h5py.File('./data/VPT_L63/minimum='+str(dt)+'-Nres'+str(N_units)+'-Ensemble'+str(ensemble))
minimum       = np.array(hf.get('minimum'))
hf.close()

#hf       = h5py.File('./data/MFE/minimum='+str(dt)+'-Nres'+str(N_units)+'-Ensemble'+str(ensemble)+'-dim'+str(dim)+'-noise') # With Noise
# hf       = h5py.File('./data/MFE/minimum='+str(dt)+'-Nres'+str(N_units)+'-Ensemble'+str(ensemble)+'-dim'+str(dim)) # Without noise
# minimum       = np.array(hf.get('minimum'))
# hf.close()


for i in range(ensemble):
    print('Realization    :',i+1)

    esn.rho      = 10**minimum[i,0]
    esn.epsilon  = minimum[i,1]
    esn.sigma_in = minimum[i,2]
    seed= i+1
    rnd = np.random.RandomState(seed)
    
    
    Win =  esn.gen_input_matrix(seed)
    W =  esn.gen_reservoir_matrix(seed)
    
    Xa, Wout_p, LHS, RHS =  esn.train(U_washout, U_tv, Y_tv,Win,W)
    
    Woutt[i] = Wout_p
    
    Winn    += [Win] 
    Ws      += [W]
    Xa_cc   += [Xa]
    LHS_cc  += [LHS]
    RHS_cc  += [RHS]   
    

Realization    : 1
Realization    : 2
Realization    : 3
Realization    : 4
Realization    : 5


# Quantum Reservoir Computing - Initialization

In [9]:
# Defining Qbit and Cbits
n              = 9 # 9 Qubits and classical
Nres           = 2**n # Total Reservoir Size
qubits         = n

# Required Input Parameters
N_units_q      = Nres #neurons or N_res in paper
bias_in        = np.array([np.mean(np.abs((U_data-u_mean)/norm))])  # Input bias, b_in to break the inherent symmetry of ESN Structure
bias_out       = np.array([1]) #output bias 

# Hyperparameters
tikh_q        = np.array([1e-12])  # Tikhonov Param (Beta),  adds a regularization or weight decay term that penalizes large Wout values 
sigma_in_q    = 1
rho_q         = 1.0 # [Spectral Radius]
epsilon_q     = 0.05 # Leaking Rate

# Ensembles
qu_ensemble   = 1

# Training 

In [10]:
Configurations = ['Linear+Full+FullSymmetric','Linear+Linear+Linear','None+Linear+Linear','None+Full+FullSymmetric','None+Products+Linear']
config_list    = ['config1','config2','config3','config4','config5']
qc_type        = config_list[3]
config         = 4 # for initializing class its +1 of the list above

emulator_list  = ['sv_sim','qasm_sim']
emulator       = emulator_list[1]

method_list    = ['sv','shots1k','shots5k','shots10k','shots20k','shots30k','shots40k','shots50k','shots75k','shots100k','shots200k','shots1']
shot_list      = [None,1000,5000,10000,20000,30000,40000,50000,75000,100000,200000,400000,1]

## SHOT INPUT
shot_index     = 1

method = method_list[shot_index]
total_shots  = shot_list[shot_index]


shots = total_shots

snapshots = 1
print('Configuration:',qc_type,'Emulator:',emulator,'Total shots:',total_shots,'Individual Shots:',shots)

# Initialize Class
QESN   = QuantumReservoirNetwork(rho_q,epsilon_q,sigma_in_q,tikh_q,bias_in,bias_out,qubits,N_units_q,dim,config,emulator,shots,snapshots)

# Parameterized or not
QESN.method_qc(parameterized=False)

# Generate parameterized quantum circuit for a given configuration
# QESN.gen_param_quantumcircuit()

quantum_training = True
run_training     = True
save_training    = False
load_training    = False

Xa_qq    = []
Woutt_qq    = np.zeros(((qu_ensemble, QESN.N_units+1,dim)))
LHS_qq   = []
RHS_qq   = []
alpha_qc = []

eps_fix  = 0.05
tikh_q   = np.array([1e-12])
eps_q_list = np.linspace(0.05,0.3,qu_ensemble)
eps_q_list = np.ones(len(eps_q_list))*eps_fix
alpha_range    = 4*np.pi

start = timer()

if quantum_training:
    for i in range(qu_ensemble):
        
        print('Network',i+1,' out of ',qu_ensemble)
        seed           = i
        
        # Quantum Variational Part
        QESN.epsilon_q = eps_q_list[i]

        alpha_q          = QESN.gen_random_unitary(i,alpha_range)
        alpha_qc        += [alpha_q]


        if load_training:
            print('Loading existing Wout matrix')
            #hf       = h5py.File('./data/MFE/Woutt_qq='+str(dt)+'-Nres'+str(N_units_q)+'-Ensemble'+str(ensemble)+'-dim'+str(dim)+'-epsilon_q'+str(eps_fix)+'-tikh'+str(tikh_q)+'script')#+'-noise')
            #hf        = h5py.File('./data/MFE/Woutt_qq='+str(dt)+'-Nres'+str(N_units)+'-Ensemble'+str(qu_ensemble)+'-dim'+str(dim)+'-epsilon_q'+str(eps_fix)+'-tikh'+str(tikh_q)+'-scaling'+str(scaling)+'-noise'+str(noise)+'-'+str(method)+str(qc_type))
            hf       = h5py.File('./data/VPT_L63/Woutt_qq='+str(dt)+'-Nres'+str(N_units)+'-Ensemble'+str(ensemble)+'-dim'+str(dim)+'-epsilon_q'+str(eps_fix)+'-tikh'+str(tikh_q)+'-scaling'+str(scaling)+'-noise'+str(noise)+'-'+str(method)+str(qc_type))
            #hf      = h5py.File('./data/{}_New/Woutt_qq='.format(save_fold)+str(dt)+'-Nres'+str(N_units)+'-Ensemble'+str(qu_ensemble)+'-dim'+str(dim)+'-epsilon_q'+str(eps_fix)+'-tikh'+str(tikh_q)+'-scaling'+str(scaling)+'-noise'+str(noise)+'-'+str(method)+str(qc_type))
            #hf      = h5py.File('./data/{}/Woutt_qq='.format(save_fold)+str(dt)+'-Nres'+str(N_units)+'-Ensemble'+str(qu_ensemble)+'-dim'+str(dim)+'-epsilon_q'+str(eps_fix)+'-tikh'+str(tikh_q)+'-scaling'+str(scaling)+'-noise'+str(noise)+'-'+str(method)+str(qc_type))
            
            Woutt_qq       = np.array(hf.get('Woutt_qq'))
            hf.close()
        

        if run_training:
            print('Training...')
            Xa_qc, Woutt_qc, LHS_qc, RHS_qc = QESN.quantum_training(U_washout, U_tv, Y_tv, alpha_qc[i])
                    
            Xa_qq       += [Xa_qc]
            Woutt_qq[i]  = Woutt_qc # saving Output matrix as array 
            LHS_qq      += [LHS_qc]
            RHS_qq      += [RHS_qc]
                
alpha_qq = alpha_qc
# Woutt_qq = Woutt_qq

if load_training:
    Woutt_qq = np.reshape(Woutt_qq,[qu_ensemble, QESN.N_units+1,dim]) # If loading previously saved Woutt , need to reshape 4D to 2D

end = timer()

training_time = (end-start)
print(training_time)


if save_training:
    fln      = ('./data/{}/Woutt_qq='.format(save_fold)+str(dt)+'-Nres'+str(N_units)+'-Ensemble'+str(qu_ensemble)+'-dim'+str(dim)+'-epsilon_q'+str(eps_fix)+'-tikh'+str(tikh_q)+'-scaling'+str(scaling)+'-noise'+str(noise)+'-'+str(method)+str(qc_type))
    hf       = h5py.File(fln,'w')
    hf.create_dataset('Woutt_qq',data=Woutt_qq)
    hf.close()


Configuration: config4 Emulator: qasm_sim Total shots: 1000 Individual Shots: 1000
Network 1  out of  1
Training...


# Predictions 

In [ ]:
## MODIFY Post processing functionss
quantum_plots = True
N_test   = dim   #number of intervals in the test set
N_tstart = N_washout          # where the first test interval starts
N_intt   = 15*N_lyap          # length of each test set interval
N_fwd    = 1*N_lyap

len_open = N_washout          # Should be < N_washout because of implementation
len_closed = 10*N_lyap

factor = 100*N_lyap           # statistical predictions

Y_tp      =  []
Yh_tp     =  []
Yh_t_qp   =  []
PH_plot_p  =  []
PH_plot_pq =  []
PH_series  = []
PH_series_q  = []

time_series = np.array([0]) # relevant for MFE

for j in range(len(time_series)):
    num_series = time_series[j]
    print('For time series',num_series)

    for i in range(qu_ensemble):
        seed  = i+1
        Win   =  esn.gen_input_matrix(seed)
        W     =  esn.gen_reservoir_matrix(seed)

        if quantum_plots:
            U_wash, Uh_wash , Y_t , Yh_t , PH_plot , Uh_wash_q, Yh_t_q , PH_plot_q = lorenz63_timeseries_plot(N_test,N_tstart,N_intt,N_fwd,j,UU,U,U_test,N_washout,N_lyap,ensemble,N_t,minimum,esn,Win,W,QESN,Woutt,Woutt_qq,alpha_qq,plot=True,quantum=True)
            
            Y_tp       +=  [Y_t]
            Yh_tp      +=  [Yh_t]
            Yh_t_qp    +=  [Yh_t_q]
            PH_plot_p  += [PH_plot[i]]
            PH_plot_pq += [PH_plot_q[i]]
            
        else:
            U_wash, Uh_wash , Y_t , Yh_t , PH_plot , Uh_wash_q, Yh_t_q , PH_plot_q = lorenz63_timeseries_plot(N_test,N_tstart,N_intt,N_fwd,j,UU,U,U_test,N_washout,N_lyap,ensemble,N_t,minimum,esn,Win,W,QESN,Woutt,Woutt_qq,alpha_qq,plot=True,quantum=False)
            Y_tp      +=  [Y_t]
            Yh_tp     +=  [Yh_t]
            PH_plot_p += [PH_plot[i]]
            
    PH_series   += [PH_plot_p]
    PH_series_q += [PH_plot_pq]
            
